# DBSCAN 密度聚类算法

**核心概念**: DBSCAN (Density-Based Spatial Clustering of Applications with Noise) 是一种基于密度的聚类算法，能够发现任意形状的簇并自动识别噪声点

## 算法原理

DBSCAN 基于两个核心参数定义"密度":

- **eps (epsilon)**: 邻域半径，定义一个点的邻域范围
- **min_samples**: 成为核心点所需的最少邻居数

### 点的分类

| 类型 | 定义 |
|------|------|
| **核心点 (Core Point)** | eps 邻域内包含至少 min_samples 个点 |
| **边界点 (Border Point)** | 不是核心点，但在某个核心点的 eps 邻域内 |
| **噪声点 (Noise Point)** | 既不是核心点也不是边界点，标签为 -1 |

### 算法步骤

1. 对每个未访问的点，检查其 eps 邻域
2. 如果邻域内点数 >= min_samples，则为核心点，创建新簇
3. 递归地将所有密度可达的点加入该簇
4. 处理完所有点后，未被分配的点标记为噪声

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN, KMeans
from sklearn.datasets import make_moons, make_blobs, make_circles
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import silhouette_score

# 设置随机种子
np.random.seed(42)

# 配置 matplotlib
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False

## 1. 月牙形数据集演示

月牙形数据是展示 DBSCAN 优势的经典例子，K-Means 无法正确处理这种非凸形状。

In [ ]:
# 生成月牙形数据
X_moons, y_moons = make_moons(n_samples=1000, noise=0.05, random_state=42)

print(f"数据集形状: {X_moons.shape}")
print(f"真实簇数: {len(np.unique(y_moons))}")

# 可视化原始数据
plt.figure(figsize=(10, 5))
plt.scatter(X_moons[:, 0], X_moons[:, 1], c=y_moons, cmap='viridis', s=20, alpha=0.7)
plt.title('月牙形数据集 (真实标签)')
plt.xlabel('特征 1')
plt.ylabel('特征 2')
plt.colorbar(label='簇标签')
plt.tight_layout()
plt.show()

In [ ]:
# 使用 DBSCAN 聚类
dbscan = DBSCAN(eps=0.2, min_samples=5)
labels_dbscan = dbscan.fit_predict(X_moons)

# 统计结果
n_clusters = len(set(labels_dbscan)) - (1 if -1 in labels_dbscan else 0)
n_noise = (labels_dbscan == -1).sum()

print(f"发现的簇数: {n_clusters}")
print(f"噪声点数量: {n_noise} ({n_noise/len(labels_dbscan)*100:.1f}%)")
print(f"核心样本数: {len(dbscan.core_sample_indices_)}")

In [ ]:
# 对比 DBSCAN 和 K-Means
kmeans = KMeans(n_clusters=2, n_init=10, random_state=42)
labels_kmeans = kmeans.fit_predict(X_moons)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# 真实标签
axes[0].scatter(X_moons[:, 0], X_moons[:, 1], c=y_moons, cmap='viridis', s=20, alpha=0.7)
axes[0].set_title('真实标签')
axes[0].set_xlabel('特征 1')
axes[0].set_ylabel('特征 2')

# K-Means (失败)
axes[1].scatter(X_moons[:, 0], X_moons[:, 1], c=labels_kmeans, cmap='viridis', s=20, alpha=0.7)
axes[1].scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1],
                c='red', marker='X', s=200, edgecolors='k', linewidths=2)
axes[1].set_title('K-Means (无法正确分离)')
axes[1].set_xlabel('特征 1')
axes[1].set_ylabel('特征 2')

# DBSCAN (成功)
colors = ['#1f77b4', '#ff7f0e', '#2ca02c']
for label in np.unique(labels_dbscan):
    if label == -1:
        # 噪声点用黑色 X 标记
        axes[2].scatter(X_moons[labels_dbscan == label, 0], 
                       X_moons[labels_dbscan == label, 1],
                       c='black', marker='x', s=30, label='噪声')
    else:
        axes[2].scatter(X_moons[labels_dbscan == label, 0],
                       X_moons[labels_dbscan == label, 1],
                       c=colors[label % len(colors)], s=20, alpha=0.7,
                       label=f'簇 {label}')
axes[2].set_title('DBSCAN (正确分离)')
axes[2].set_xlabel('特征 1')
axes[2].set_ylabel('特征 2')
axes[2].legend()

plt.tight_layout()
plt.show()

## 2. 核心点、边界点与噪声点可视化

In [ ]:
# 识别核心点和边界点
core_mask = np.zeros(len(X_moons), dtype=bool)
core_mask[dbscan.core_sample_indices_] = True
noise_mask = labels_dbscan == -1
border_mask = ~core_mask & ~noise_mask

print(f"核心点: {core_mask.sum()}")
print(f"边界点: {border_mask.sum()}")
print(f"噪声点: {noise_mask.sum()}")

# 可视化三类点
plt.figure(figsize=(10, 6))

# 核心点
plt.scatter(X_moons[core_mask, 0], X_moons[core_mask, 1],
            c=labels_dbscan[core_mask], cmap='viridis', s=50, alpha=0.7,
            edgecolors='k', linewidths=0.5, label='核心点')

# 边界点
plt.scatter(X_moons[border_mask, 0], X_moons[border_mask, 1],
            c=labels_dbscan[border_mask], cmap='viridis', s=30, alpha=0.5,
            marker='s', label='边界点')

# 噪声点
plt.scatter(X_moons[noise_mask, 0], X_moons[noise_mask, 1],
            c='red', marker='x', s=50, label='噪声点')

plt.title('DBSCAN 点分类: 核心点、边界点、噪声点')
plt.xlabel('特征 1')
plt.ylabel('特征 2')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 3. 参数敏感性分析

DBSCAN 的结果对 `eps` 和 `min_samples` 参数非常敏感。

In [ ]:
# eps 参数敏感性
eps_values = [0.05, 0.1, 0.2, 0.3, 0.5, 0.8]

fig, axes = plt.subplots(2, 3, figsize=(15, 9))
axes = axes.ravel()

for i, eps in enumerate(eps_values):
    db = DBSCAN(eps=eps, min_samples=5)
    labels = db.fit_predict(X_moons)
    
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise = (labels == -1).sum()
    
    # 绘制结果
    unique_labels = set(labels)
    colors = plt.cm.Spectral(np.linspace(0, 1, len(unique_labels)))
    
    for label, color in zip(unique_labels, colors):
        if label == -1:
            axes[i].scatter(X_moons[labels == label, 0], X_moons[labels == label, 1],
                           c='black', marker='x', s=30)
        else:
            axes[i].scatter(X_moons[labels == label, 0], X_moons[labels == label, 1],
                           c=[color], s=20, alpha=0.7)
    
    axes[i].set_title(f'eps={eps}\n簇数={n_clusters}, 噪声={n_noise}')
    axes[i].set_xlabel('特征 1')
    axes[i].set_ylabel('特征 2')

plt.suptitle('eps 参数对 DBSCAN 聚类的影响 (min_samples=5)', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# min_samples 参数敏感性
min_samples_values = [2, 3, 5, 10, 15, 20]

fig, axes = plt.subplots(2, 3, figsize=(15, 9))
axes = axes.ravel()

for i, min_samples in enumerate(min_samples_values):
    db = DBSCAN(eps=0.2, min_samples=min_samples)
    labels = db.fit_predict(X_moons)
    
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise = (labels == -1).sum()
    
    unique_labels = set(labels)
    colors = plt.cm.Spectral(np.linspace(0, 1, len(unique_labels)))
    
    for label, color in zip(unique_labels, colors):
        if label == -1:
            axes[i].scatter(X_moons[labels == label, 0], X_moons[labels == label, 1],
                           c='black', marker='x', s=30)
        else:
            axes[i].scatter(X_moons[labels == label, 0], X_moons[labels == label, 1],
                           c=[color], s=20, alpha=0.7)
    
    axes[i].set_title(f'min_samples={min_samples}\n簇数={n_clusters}, 噪声={n_noise}')
    axes[i].set_xlabel('特征 1')
    axes[i].set_ylabel('特征 2')

plt.suptitle('min_samples 参数对 DBSCAN 聚类的影响 (eps=0.2)', fontsize=14)
plt.tight_layout()
plt.show()

## 4. eps 参数选择方法: K-距离图

K-距离图是选择合适 eps 的常用方法:

1. 对每个点，计算到第 K 近邻的距离 (K 通常设为 min_samples)
2. 将距离从小到大排序并绘图
3. 图中"肘部"对应的距离值是合适的 eps

In [ ]:
def plot_k_distance(X, k=5):
    """
    绘制 K-距离图以辅助选择 eps 参数
    
    参数:
        X: 特征数据
        k: 近邻数 (通常设为 min_samples)
    """
    # 计算每个点到第 k 个近邻的距离
    neighbors = NearestNeighbors(n_neighbors=k)
    neighbors.fit(X)
    distances, _ = neighbors.kneighbors(X)
    
    # 取第 k 个近邻的距离并排序
    k_distances = np.sort(distances[:, k-1])
    
    # 绘图
    plt.figure(figsize=(10, 5))
    plt.plot(range(len(k_distances)), k_distances, 'b-', linewidth=1)
    plt.xlabel('样本索引 (按距离排序)', fontsize=12)
    plt.ylabel(f'第 {k} 近邻距离', fontsize=12)
    plt.title(f'K-距离图 (K={k})\n"肘部"处的距离值建议作为 eps', fontsize=14)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    return k_distances

# 绘制 K-距离图
k_distances = plot_k_distance(X_moons, k=5)

# 打印一些参考值
percentiles = [50, 75, 90, 95]
for p in percentiles:
    val = np.percentile(k_distances, p)
    print(f"{p}% 分位数距离: {val:.4f}")

## 5. 同心圆数据集

另一个 K-Means 无法处理但 DBSCAN 可以正确聚类的例子。

In [ ]:
# 生成同心圆数据
X_circles, y_circles = make_circles(n_samples=1000, factor=0.5, noise=0.05, random_state=42)

# DBSCAN 聚类
dbscan_circles = DBSCAN(eps=0.15, min_samples=5)
labels_circles = dbscan_circles.fit_predict(X_circles)

# K-Means 聚类
kmeans_circles = KMeans(n_clusters=2, n_init=10, random_state=42)
labels_kmeans_circles = kmeans_circles.fit_predict(X_circles)

# 可视化
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].scatter(X_circles[:, 0], X_circles[:, 1], c=y_circles, cmap='viridis', s=20)
axes[0].set_title('真实标签')

axes[1].scatter(X_circles[:, 0], X_circles[:, 1], c=labels_kmeans_circles, cmap='viridis', s=20)
axes[1].set_title('K-Means (失败)')

for label in np.unique(labels_circles):
    if label == -1:
        axes[2].scatter(X_circles[labels_circles == label, 0],
                       X_circles[labels_circles == label, 1],
                       c='black', marker='x', s=30, label='噪声')
    else:
        axes[2].scatter(X_circles[labels_circles == label, 0],
                       X_circles[labels_circles == label, 1],
                       s=20, alpha=0.7, label=f'簇 {label}')
axes[2].set_title('DBSCAN (成功)')
axes[2].legend()

for ax in axes:
    ax.set_xlabel('特征 1')
    ax.set_ylabel('特征 2')

plt.tight_layout()
plt.show()

## 6. 异常检测应用

DBSCAN 的噪声点检测功能使其天然适合异常检测。

In [ ]:
# 生成带异常点的数据
X_normal, _ = make_blobs(n_samples=500, centers=2, cluster_std=0.5, random_state=42)

# 添加一些异常点
n_outliers = 30
outliers = np.random.uniform(low=-5, high=5, size=(n_outliers, 2))
X_with_outliers = np.vstack([X_normal, outliers])

# 真实标签: 0, 1 为正常簇, 2 为异常
y_true_anomaly = np.concatenate([np.zeros(250), np.ones(250), np.full(n_outliers, 2)])

print(f"总样本数: {len(X_with_outliers)}")
print(f"正常样本: {len(X_normal)}")
print(f"异常样本: {n_outliers}")

In [ ]:
# 使用 DBSCAN 进行异常检测
dbscan_anomaly = DBSCAN(eps=0.5, min_samples=5)
labels_anomaly = dbscan_anomaly.fit_predict(X_with_outliers)

# 噪声点即为检测到的异常
detected_outliers = labels_anomaly == -1

print(f"\n检测结果:")
print(f"发现的簇数: {len(set(labels_anomaly)) - 1}")
print(f"检测到的异常点数: {detected_outliers.sum()}")

# 计算检测准确率
true_outliers = y_true_anomaly == 2
correct_detection = (detected_outliers & true_outliers).sum()
print(f"正确检测的异常: {correct_detection}/{n_outliers}")

# 可视化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 真实分布
axes[0].scatter(X_normal[:, 0], X_normal[:, 1], c='blue', s=20, alpha=0.7, label='正常点')
axes[0].scatter(outliers[:, 0], outliers[:, 1], c='red', marker='X', s=80, label='真实异常')
axes[0].set_title('真实数据分布')
axes[0].legend()

# DBSCAN 检测结果
normal_detected = X_with_outliers[~detected_outliers]
anomaly_detected = X_with_outliers[detected_outliers]

axes[1].scatter(normal_detected[:, 0], normal_detected[:, 1], 
               c=labels_anomaly[~detected_outliers], cmap='viridis', s=20, alpha=0.7, label='正常簇')
axes[1].scatter(anomaly_detected[:, 0], anomaly_detected[:, 1],
               c='red', marker='X', s=80, label='检测到的异常')
axes[1].set_title('DBSCAN 异常检测结果')
axes[1].legend()

for ax in axes:
    ax.set_xlabel('特征 1')
    ax.set_ylabel('特征 2')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. 新样本预测

DBSCAN 没有内置的 `predict` 方法。常用解决方案是使用 KNN 分类器对已聚类的核心点进行训练。

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# 使用 DBSCAN 训练集
dbscan_train = DBSCAN(eps=0.2, min_samples=5)
train_labels = dbscan_train.fit_predict(X_moons)

# 提取非噪声点用于训练 KNN
non_noise_mask = train_labels != -1
X_train_knn = X_moons[non_noise_mask]
y_train_knn = train_labels[non_noise_mask]

# 训练 KNN 分类器
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_knn, y_train_knn)

# 生成新样本
X_new = np.array([
    [0.5, 0.3],   # 应该属于簇 0
    [1.0, -0.3],  # 应该属于簇 1
    [0.5, 1.0],   # 可能是异常点
])

# 预测新样本
new_labels = knn.predict(X_new)
new_proba = knn.predict_proba(X_new)

print("新样本预测结果:")
for i, (x, label, proba) in enumerate(zip(X_new, new_labels, new_proba)):
    print(f"  样本 {i}: 坐标={x}, 预测簇={label}, 置信度={proba.max():.2f}")

# 可视化
plt.figure(figsize=(10, 6))
plt.scatter(X_moons[:, 0], X_moons[:, 1], c=train_labels, cmap='viridis', s=20, alpha=0.5)
plt.scatter(X_new[:, 0], X_new[:, 1], c='red', marker='*', s=300, edgecolors='k', linewidths=2,
            label='新样本')
for i, (x, label) in enumerate(zip(X_new, new_labels)):
    plt.annotate(f'预测: {label}', xy=x, xytext=(x[0]+0.1, x[1]+0.1), fontsize=10)
plt.title('DBSCAN + KNN 预测新样本')
plt.xlabel('特征 1')
plt.ylabel('特征 2')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 总结

### DBSCAN 优点

1. **无需指定簇数**: 自动发现簇的数量
2. **发现任意形状**: 可以处理非凸形状的簇
3. **噪声检测**: 自动识别异常点
4. **对异常值鲁棒**: 不受离群点影响

### DBSCAN 局限性

1. **参数敏感**: eps 和 min_samples 对结果影响大
2. **密度不均匀**: 难以处理不同密度的簇
3. **高维数据**: 在高维空间中距离度量失效
4. **无法预测**: 需要额外方法处理新样本

### 参数选择建议

| 参数 | 建议 |
|------|------|
| eps | 使用 K-距离图找"肘部" |
| min_samples | 通常设为 2*维度 或 维度+1 |

### 适用场景

- 空间数据聚类 (地理位置、图像分割)
- 异常检测 (欺诈检测、故障检测)
- 非凸形状数据 (月牙、同心圆等)
- 噪声数据处理